#**Temporal Separation with Entropy Regularization for Knowledge Distillation**

#**Setup**

---
**Install Libraries**

In [ ]:
!pip install snntorch dagshub mlflow pynvml --quiet

---
**GitHub Code**

In [ ]:
from google.colab import userdata
import os

# Sets environ variables for GitHub
os.environ['GITHUB_TOKEN'] = userdata.get('GITHUB_TOKEN')
os.environ['USER'] = userdata.get('USER')

# Clones the repo and changes dir
!git clone https://${GITHUB_TOKEN}@github.com/${USER}/tser-kd.git
%cd tser-kd/

fatal: destination path 'tser-kd' already exists and is not an empty directory.
/content/tser-kd


---
**Set Seed for Experiment**

In [ ]:
from tser_kd.utils import setup_seed

setup_seed(42)

Random seed: 42


---
**Device Selection**

In [ ]:
import torch

# Selects the device for the experiment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

---
**MLFlow Setup**

In [ ]:
import mlflow
from mlflow import MlflowClient
import dagshub

# Sets environ variables for MLFlow
os.environ['MLFLOW_TRACKING_USERNAME'] = userdata.get('USER')
os.environ['MLFLOW_TRACKING_PASSWORD'] = userdata.get('MLFLOW_TRACKING_PASSWORD')

# Init DagsHub
dagshub.init(repo_owner='matteogianferrari', repo_name='tser-kd', mlflow=True)
TRACKING_URI = "https://dagshub.com/matteogianferrari/tser-kd.mlflow"

# Sets MLFlow tracking URI
mlflow.set_tracking_uri(TRACKING_URI)

# Sets MLFLow experiment name
experiment_name = "TSER-KD"

Accessing as matteogianferrari

Initialized MLflow to track repo "matteogianferrari/tser-kd"

Repository matteogianferrari/tser-kd initialized!

#**Hyperparameters**

In [ ]:
# Hyperparamter dictionary
h_dict = {
    "TAU": 5.0, "ALPHA": 0.7, "GAMMA": 1e-2,                # Loss
    "BETA": 0.5, "V_th": 1.0,                               # Leaky Neuron
    "MAX_EPOCHS": 300, "BATCH_SIZE": 32,                    # Training
    "LR_SCHEDULER": "CosineAnnealingLR", "BASE_LR": 1e-4,    # LR
    "ES_PATIENCE": 300, "ES_DELTA": 1e-4,                    # Early Stopping
    "OPTIMIZER": "AdamW", "WEIGHT_DECAY": 5e-4,             # Optimizer
    "HARDWARE": "A100",                                     # GPU
    "ENCODER": "Static", "T": 2,                            # Encoder
    "AUTO_AUG": True, "CUTOUT": True,                       # Dataset
}

#**CIFAR10 Dataset**

---
**Data Loaders Creation**

In [ ]:
from tser_kd.dataset import load_cifar10_data
from torch.utils.data import DataLoader


train_dataset, val_dataset, num_classes = load_cifar10_data(auto_aug=h_dict['AUTO_AUG'], cutout=h_dict['CUTOUT'])

# Creates the train and test DataLoaders
train_loader = DataLoader(train_dataset, batch_size=h_dict['BATCH_SIZE'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=h_dict['BATCH_SIZE'], shuffle=False, num_workers=2)

#**Teacher and Student Models**

---
**Models Creation**

In [ ]:
from tser_kd.model.student import make_student_model
from tser_kd.model.teacher import make_teacher_model

# ANN
t_state_dict = torch.load("data/teacher_models/resnet34_ft_9708.pth", map_location="cpu")
t_model = make_teacher_model(
    arch='resnet-34',
    in_channels=3,
    num_classes=num_classes,
    device=device,
    state_dict=t_state_dict
)

# SNN
s_model = make_student_model(
    arch='sresnet-19',
    in_channels=3,
    num_classes=num_classes,
    beta=h_dict['BETA'],
    device=device
)

In [ ]:
@torch.no_grad()
def load_sresnet18_backbone_into_sresnet19(res18, model):
    """
    Copies ImageNet-pretrained ResNet-18 weights into your CIFAR ResNet19 backbone.
    Leaves stem and FC layers as-is. Returns a small report dict.
    """

    def copy_conv(dst, src):
        assert dst.weight.shape == src.weight.shape, f"conv shape mismatch: {dst.weight.shape} vs {src.weight.shape}"
        dst.weight.copy_(src.weight)
        if getattr(dst, "bias", None) is not None and getattr(src, "bias", None) is not None:
            dst.bias.copy_(src.bias)

    def copy_bn(dst, src):
        assert dst.weight.shape == src.weight.shape, f"bn shape mismatch: {dst.weight.shape} vs {src.weight.shape}"
        dst.weight.copy_(src.weight)
        dst.bias.copy_(src.bias)
        dst.running_mean.copy_(src.running_mean)
        dst.running_var.copy_(src.running_var)
        if hasattr(dst, "num_batches_tracked") and hasattr(src, "num_batches_tracked"):
            dst.num_batches_tracked.copy_(src.num_batches_tracked)

    def copy_block(dst_block, src_block, copy_shortcut=False):
        copy_conv(dst_block.t_conv_bn1.layer, src_block.t_conv_bn1.layer); copy_bn(dst_block.t_conv_bn1.batch_norm, src_block.t_conv_bn1.batch_norm)
        copy_conv(dst_block.t_conv_bn2.layer, src_block.t_conv_bn2.layer); copy_bn(dst_block.t_conv_bn2.batch_norm, src_block.t_conv_bn2.batch_norm)
        if copy_shortcut and (dst_block.shortcuts is not None) and (src_block.shortcuts is not None):
            # conv + bn in the 1x1 projection
            copy_conv(dst_block.shortcuts.layer, src_block.shortcuts.layer)
            copy_bn(dst_block.shortcuts.batch_norm,  src_block.shortcuts.batch_norm)

    # ---- Mappings ----
    # block1 (128 ch): use layer2[1] (128->128 stride 1) for all three
    for i in range(3):
        copy_block(model.stages[0][i], res18.stages[1][1], copy_shortcut=False)

    # block2 (256 ch): first with downsample, then two stride-1 blocks
    copy_block(model.stages[1][0], res18.stages[2][0], copy_shortcut=True)   # 128->256, stride 2
    copy_block(model.stages[1][1], res18.stages[2][1], copy_shortcut=False)  # 256->256
    copy_block(model.stages[1][2], res18.stages[2][1], copy_shortcut=False)  # duplicate

    # block3 (512 ch): first with downsample, then stride-1
    copy_block(model.stages[2][0], res18.stages[3][0], copy_shortcut=True)   # 256->512, stride 2
    copy_block(model.stages[2][1], res18.stages[3][1], copy_shortcut=False)  # 512->512

    report = {
        "copied_from_resnet18": {
            "block1": ["layer2[1] → block1[0,1,2]"],
            "block2": ["layer3[0] → block2[0] (with shortcut)",
                       "layer3[1] → block2[1,2]"],
            "block3": ["layer4[0] → block3[0] (with shortcut)",
                       "layer4[1] → block3[1]"],
        },
        "left_random_init": ["stem conv/bn", "fc1", "fc2"],
    }
    return report


s_state_dict = torch.load("data/student_models/sresnet18_pt_9078.pth", map_location="cpu")
sres18 = make_student_model(
    arch='sresnet-18',
    in_channels=3,
    num_classes=num_classes,
    beta=h_dict["BETA"],
    device=device,
    state_dict=s_state_dict
)

load_sresnet18_backbone_into_sresnet19(sres18, s_model)

{'copied_from_resnet18': {'block1': ['layer2[1] → block1[0,1,2]'],
  'block2': ['layer3[0] → block2[0] (with shortcut)',
   'layer3[1] → block2[1,2]'],
  'block3': ['layer4[0] → block3[0] (with shortcut)',
   'layer4[1] → block3[1]']},
 'left_random_init': ['stem conv/bn', 'fc1', 'fc2']}

In [ ]:
# Remove the a_model and a_state_dict
# IMPORTANT TO KEEP THE EPOCH TIME LOW
del sres18, s_state_dict
torch.cuda.empty_cache()

#**Training**

In [ ]:
import torch.optim as optim
import torch.nn as nn
from tser_kd.training import EarlyStopping
from tser_kd.dataset import RateEncoder, StaticEncoder
from tser_kd.model import TSERKDLoss
from tser_kd.training.lr_scheduler import WarmupCosineLR
from tser_kd.utils import AccuracyMonitor


# Optimizer
if h_dict["OPTIMIZER"] == 'AdamW':
    optimizer = optim.AdamW(s_model.parameters(), lr=h_dict['BASE_LR'], weight_decay=h_dict['WEIGHT_DECAY'])
elif h_dict["OPTIMIZER"] == 'Adam':
    optimizer = optim.Adam(s_model.parameters(), lr=h_dict['BASE_LR'], weight_decay=h_dict['WEIGHT_DECAY'])
elif h_dict["OPTIMIZER"] == 'SGD':
    optimizer = optim.SGD(s_model.parameters(), lr=h_dict['BASE_LR'], momentum=h_dict["MOMENTUM"], weight_decay=h_dict['WEIGHT_DECAY'])

# LR scheduler
if h_dict["LR_SCHEDULER"] == 'ReduceLROnPlateau':
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=h_dict["LR_PATIENCE"], factor=h_dict["LR_FACTOR"])
elif h_dict["LR_SCHEDULER"] == 'CosineAnnealingLR':
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=h_dict["MAX_EPOCHS"])
elif h_dict["LR_SCHEDULER"] == 'StepLR':
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=h_dict["LR_STEP"], gamma=h_dict["LR_FACTOR"])
elif h_dict["LR_SCHEDULER"] == 'WarmupCosineLR':
    scheduler = WarmupCosineLR(
        optimizer=optimizer,
        warmup_epochs=h_dict["WARMUP_EPOCHS"],
        total_epochs=h_dict["MAX_EPOCHS"],
        base_lr=h_dict["BASE_LR"],
        max_lr=h_dict["MAX_LR"]
    )

# Losses
train_criterion = TSERKDLoss(alpha=h_dict["ALPHA"], gamma=h_dict["GAMMA"], tau=h_dict["TAU"])
eval_criterion = nn.CrossEntropyLoss()

# Accuracy monitor
acc_monitor = AccuracyMonitor(path="best_acc.pth")

# Early stopping
es_callback = EarlyStopping(patience=h_dict["ES_PATIENCE"], delta=h_dict["ES_DELTA"], path="best_loss.pth")

# Gradient scaler
scaler = torch.amp.GradScaler(device='cuda')

# Encoder
if h_dict["ENCODER"] == "Rate":
    encoder = RateEncoder(num_steps=h_dict["T"], gain=h_dict["GAIN"])
elif h_dict["ENCODER"] == "Static":
    encoder = StaticEncoder(num_steps=h_dict["T"])

In [ ]:
import pynvml
from tser_kd.eval import run_eval
from tser_kd.training import run_kd_train


# Sets the MLFlow experiment
mlflow.set_experiment(experiment_name)

epoch_i = 0
START_EPOCH = 0
curr_lr = optimizer.param_groups[0]["lr"]

# Train the model and log with MLFlow
with mlflow.start_run(run_id=None, log_system_metrics=True):
    for epoch_i in range(h_dict["MAX_EPOCHS"]):
        train_total_loss, train_ce_loss, train_kl_loss, train_e_reg, train_acc, epoch_time, train_batch_time = run_kd_train(
            epoch_i, train_loader, s_model, t_model, train_criterion, optimizer, device, scaler, encoder
        )

        val_loss, val_acc1, val_acc5, val_batch_time = run_eval(
            val_loader, s_model, eval_criterion, device, encoder
        )

        # Logging
        print(
            f"Time: {epoch_time:.1f}s | Train Total Loss: {train_total_loss:.4f} | Train Acc: {train_acc:.2f}% | "
            f"Val Loss: {val_loss:.4f} | Val Acc1: {val_acc1:.2f}% | Val Acc5: {val_acc5:.2f}% | LR: {curr_lr:.6f}"
        )

        mlflow.log_metrics({
            "learning_rate": curr_lr, "train_tserkd_loss": train_total_loss, "train_tsce_loss": train_ce_loss, "train_tskl_loss": train_kl_loss,
            "train_e_reg": train_e_reg, "train_acc": train_acc, "val_ce_loss": val_loss, "val_acc1": val_acc1, "val_acc5": val_acc5, "epoch_time": epoch_time,
            "train_batch_time": train_batch_time, "val_batch_time": val_batch_time
        }, step=epoch_i)

        # Updates the LR
        if h_dict["LR_SCHEDULER"] == 'ReduceLROnPlateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()

        curr_lr = optimizer.param_groups[0]["lr"]

        # ES check
        if es_callback(val_loss, epoch_i, s_model):
            break

    # Log hyperparameters
    mlflow.log_params(h_dict)

    # Log test performance
    s_model.load_state_dict(torch.load("best_acc.pth"))
    test_ce_loss, test_acc1, test_acc5, _ = run_eval(val_loader, s_model, eval_criterion, device, encoder)
    mlflow.log_metrics({"test_ce_loss": test_ce_loss, "test_acc1": test_acc1, "test_acc5": test_acc5})

2025/07/30 23:41:59 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
Epoch 1: 100%|██████████| Batch 1563/1563 , acc=19.01%, ce_loss=2.1198, e_reg=2.1959, kl_loss=1.1730, total_loss=21.1420


Time: 227.7s | Train Total Loss: 21.1420 | Train Acc: 19.01% | Val Loss: 2.1498 | Val Acc1: 30.25% | Val Acc5: 86.08% | LR: 0.000100


Epoch 2: 100%|██████████| Batch 1563/1563 , acc=26.34%, ce_loss=1.9539, e_reg=2.0644, kl_loss=1.0536, total_loss=19.0036


Time: 226.1s | Train Total Loss: 19.0036 | Train Acc: 26.34% | Val Loss: 2.3168 | Val Acc1: 35.73% | Val Acc5: 90.08% | LR: 0.000100


Epoch 3: 100%|██████████| Batch 1563/1563 , acc=30.39%, ce_loss=1.8777, e_reg=2.0141, kl_loss=0.9954, total_loss=17.9635


...

Epoch 188: 100%|██████████| Batch 1563/1563 , acc=82.16%, ce_loss=0.6686, e_reg=1.1515, kl_loss=0.1850, total_loss=3.4260


Time: 227.6s | Train Total Loss: 3.4260 | Train Acc: 82.16% | Val Loss: 0.6854 | Val Acc1: 88.47% | Val Acc5: 99.63% | LR: 0.000031


Epoch 189: 100%|██████████| Batch 1563/1563 , acc=82.34%, ce_loss=0.6648, e_reg=1.1479, kl_loss=0.1814, total_loss=3.3625


Time: 227.9s | Train Total Loss: 3.3625 | Train Acc: 82.34% | Val Loss: 0.6913 | Val Acc1: 88.47% | Val Acc5: 99.73% | LR: 0.000031


Epoch 190:  95%|█████████▍| Batch 1484/1563 , acc=82.52%, ce_loss=0.6652, e_reg=1.1522, kl_loss=0.1829, total_loss=3.3892

In [ ]:
torch.save(s_model.state_dict(), "9229.pth")

In [ ]:
for batch_i in range(len(s_logits_list)):
    T, B, K = s_logits_list[batch_i].shape

    if B != 256:
        # Remove this tensor
        s_logits_list.pop(batch_i)

In [ ]:
for batch_i in range(len(t_logits_list)):
    B, K = t_logits_list[batch_i].shape

    if B != 256:
        # Remove this tensor
        t_logits_list.pop(batch_i)

In [ ]:
# Convert list to tensor
s_logits_tensor = torch.stack(s_logits_list)
t_logits_tensor = torch.stack(t_logits_list)

print(s_logits_tensor.shape)
print(t_logits_tensor.shape)

In [ ]:
# Replicate the logits of the teacher along the temporal axis
t_logits_tensor = t_logits_tensor.unsqueeze(1).repeat(1, s_logits_tensor.shape[1], 1, 1)

print(s_logits_tensor.shape)
print(t_logits_tensor.shape)

In [ ]:
# Save these 2 objects in pickle
import pickle

with open('s_logits_tensor.pkl', 'wb') as f:
    pickle.dump(s_logits_tensor, f)

with open('t_logits_tensor.pkl', 'wb') as f:
    pickle.dump(t_logits_tensor, f)

In [ ]:
logits_t1 = s_logits_tensor[-1][0][0]

In [ ]:
logits_t2 = s_logits_tensor[-1][1][0]

In [ ]:
t_logits = t_logits_tensor[-1][0][0]

In [ ]:
# Perform softmax with temperature over the logits
softmax_t1 = torch.softmax(logits_t1 / h_dict['TAU'], dim=0)
softmax_t2 = torch.softmax(logits_t2 / h_dict['TAU'], dim=0)
softmax_t = torch.softmax(t_logits / h_dict['TAU'], dim=0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_prob_tensors(tensors, class_names=None, alpha=0.4):
    """
    Plot multiple 1‑D tensors (probability distributions) for the same set of classes,
    overlaying semi‑transparent bars so overlaps are easy to spot.

    Parameters
    ----------
    tensors : list or iterable of 1‑D arrays
        Each entry must be a length‑N iterable (N = #classes) of probabilities.
    class_names : list of str, optional
        Labels for the x‑axis. If None, classes are numbered 1..N.
    alpha : float, optional
        Transparency for the bars (0 = fully transparent, 1 = opaque).
    """
    tensors = [np.asarray(t) for t in tensors]
    n_classes = tensors[0].shape[0]
    for t in tensors:
        if t.shape[0] != n_classes:
            raise ValueError("All tensors must have the same length")

    if class_names is None:
        class_names = [str(i + 1) for i in range(n_classes)]

    x = np.arange(n_classes)
    width = 0.8 / len(tensors)  # keep the bars inside the class bin

    plt.figure(figsize=(8, 3))
    for i, t in enumerate(tensors):
        plt.bar(x + (i - (len(tensors) - 1) / 2) * width,
                t,
                width=width,
                label=f"tensor {i + 1}",
                alpha=alpha)

    plt.xticks(x, class_names)
    plt.ylabel("Probability")
    plt.xlabel("Class")
    plt.title("ResNet34 Probability Distribution")
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plt.show()

#,
tensor_list = [softmax_t1.cpu().detach().numpy(), softmax_t2.cpu().detach().numpy(), softmax_t.cpu().detach().numpy()]
plot_prob_tensors(tensor_list, alpha=0.5)